In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
census_df = pd.read_csv('co-est2019-alldata.csv', encoding='latin-1')
census_df = census_df[census_df.SUMLEV == 50]
census_df

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2019,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,RNETMIG2016,RNETMIG2017,RNETMIG2018,RNETMIG2019
1,50,3,6,1,1,Alabama,Autauga County,54571,54597,54773,...,4.847310,6.018182,-6.226119,-3.902226,1.970443,-1.712875,4.777171,0.849656,0.540916,4.560062
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183112,...,24.017829,16.641870,17.488579,22.751474,20.184334,17.725964,21.279291,22.398256,24.727215,24.380567
3,50,3,6,1,5,Alabama,Barbour County,27457,27455,27327,...,-5.690302,0.292676,-6.897817,-8.132185,-5.140431,-15.724575,-18.238016,-24.998528,-8.754922,-5.165664
4,50,3,6,1,7,Alabama,Bibb County,22915,22915,22870,...,1.385134,-4.998356,-3.787545,-5.797999,1.331144,1.329817,-0.708717,-3.234669,-6.857092,1.831952
5,50,3,6,1,9,Alabama,Blount County,57322,57322,57376,...,1.020788,0.208812,-1.650165,-0.347225,-2.049590,-1.338525,-1.391062,6.193562,-0.069229,1.124597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3188,50,4,8,56,37,Wyoming,Sweetwater County,43806,43806,43574,...,-17.605427,0.502513,15.035735,-4.614071,-13.140225,-13.762031,-18.484164,-23.515727,-18.743773,-17.065527
3189,50,4,8,56,39,Wyoming,Teton County,21294,21298,21296,...,-1.583464,-2.575509,1.905293,23.259519,12.508872,4.670450,1.253214,-0.171608,-12.432212,1.797445
3190,50,4,8,56,41,Wyoming,Uinta County,21118,21121,21089,...,-9.181105,-18.196975,-4.487730,-10.870861,-15.033634,-10.484550,-11.485101,-18.923455,-13.554993,-9.181105
3191,50,4,8,56,43,Wyoming,Washakie County,8533,8528,8530,...,-7.652085,-12.839390,-3.084589,-1.307811,-18.938032,0.000000,-15.204038,-15.950541,-17.246806,-8.289759


In [3]:
combine_fips = lambda a, b: '{:0>2}{:0>3}'.format(a, b)
census_df['FIPS'] = census_df.STATE.combine(census_df.COUNTY, combine_fips).astype(int)
census_df = census_df[['FIPS', 'POPESTIMATE2019']].set_index('FIPS')
fip_pop_dict = census_df.POPESTIMATE2019.to_dict()
census_df

,POPESTIMATE2019
FIPS,
1001,55869
1003,223234
1005,24686
1007,22394
1009,57826
...,...
56037,42343
56039,23464
56041,20226


In [4]:
start_date_nyt = '2020-04-12'

In [5]:
# NYTimes Data
# https://github.com/nytimes/covid-19-data

nyt_corona_df = pd.read_csv('git/covid-19-data/us-counties.csv')
nyt_corona_df.dropna(inplace=True)
nyt_corona_df.fips = nyt_corona_df.fips.astype(int)
nyt_corona_df.date = pd.to_datetime(nyt_corona_df.date)
#nyt_corona_df['Active'] = nyt_corona_df.cases-nyt_corona_df.deaths
nyt_corona_df = nyt_corona_df[nyt_corona_df.date >= start_date_nyt][['fips', 'date', 'cases']]
nyt_corona_df = nyt_corona_df.rename(columns={'fips': 'FIPS'})
#nyt_corona_df['cases_pct'] = [j/fip_pop_dict[i] for i, j in nyt_corona_df[['FIPS', 'cases']].to_numpy()]
nyt_corona_df

,FIPS,date,cases
51165,1001,2020-04-12,19
51166,1003,2020-04-12,71
51167,1005,2020-04-12,10
51168,1007,2020-04-12,16
51169,1009,2020-04-12,13
...,...,...,...
101103,56035,2020-04-29,1
101104,56037,2020-04-29,10
101105,56039,2020-04-29,64
101106,56041,2020-04-29,6


In [6]:
nyt_pivot = nyt_corona_df.pivot('FIPS', 'date').cases
nyt_pivot

date,2020-04-12,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,2020-04-22,2020-04-23,2020-04-24,2020-04-25,2020-04-26,2020-04-27,2020-04-28,2020-04-29
FIPS,,,,,,,,,,,,,,,,,,
1001,19.0,19.0,23.0,25.0,25.0,26.0,25.0,27.0,28.0,30.0,32.0,33.0,36.0,37.0,37.0,39.0,40.0,43.0
1003,71.0,78.0,87.0,98.0,102.0,103.0,109.0,114.0,117.0,123.0,132.0,143.0,147.0,154.0,161.0,168.0,171.0,173.0
1005,10.0,10.0,11.0,13.0,14.0,15.0,18.0,20.0,22.0,28.0,29.0,30.0,32.0,33.0,33.0,35.0,37.0,37.0
1007,16.0,17.0,17.0,19.0,24.0,23.0,26.0,28.0,32.0,32.0,34.0,33.0,34.0,35.0,38.0,42.0,42.0,42.0
1009,13.0,15.0,16.0,17.0,18.0,20.0,20.0,21.0,22.0,26.0,29.0,31.0,31.0,31.0,34.0,34.0,34.0,36.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56035,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
56037,7.0,9.0,9.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0
56039,56.0,56.0,57.0,59.0,59.0,61.0,62.0,62.0,62.0,62.0,63.0,63.0,64.0,64.0,64.0,64.0,65.0,64.0


In [7]:
# JHU Dataset 
# https://github.com/CSSEGISandData/COVID-19
dates = [f'04-{12+i:02d}-2020' for i in range(18)]

def get_jhu_table(date):
    df = pd.read_csv(f'git/COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/{date}.csv')
    df['date'] = date
    df.date = pd.to_datetime(df.date)
    df.dropna(subset=['FIPS'], inplace=True)
    df.FIPS = df.FIPS.astype(int)
    df = df[df.FIPS.isin(census_df.index)]
    #df = df.iloc[np.unique(df.FIPS, return_index=True)[1]]
    #df['cases_pct'] = [j/fip_pop_dict[i] for i, j in df[['FIPS', 'Confirmed']].to_numpy()]
    return df
    return df[['FIPS', 'date', 'Confirmed']]
    
jhu_corona_df = pd.concat([get_jhu_table(i) for i in dates])
jhu_corona_df

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,date
0,45001,Abbeville,South Carolina,US,2020-04-12 23:18:00,34.223334,-82.461707,9,0,0,9,"Abbeville, South Carolina, US",2020-04-12
1,22001,Acadia,Louisiana,US,2020-04-12 23:18:00,30.295065,-92.414197,99,5,0,94,"Acadia, Louisiana, US",2020-04-12
2,51001,Accomack,Virginia,US,2020-04-12 23:18:00,37.767072,-75.632346,15,0,0,15,"Accomack, Virginia, US",2020-04-12
3,16001,Ada,Idaho,US,2020-04-12 23:18:00,43.452658,-116.241552,517,6,0,511,"Ada, Idaho, US",2020-04-12
4,19001,Adair,Iowa,US,2020-04-12 23:18:00,41.330756,-94.471059,1,0,0,1,"Adair, Iowa, US",2020-04-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2893,2290,Yukon-Koyukuk,Alaska,US,2020-04-30 02:32:27,65.508155,-151.390739,1,0,0,1,"Yukon-Koyukuk, Alaska, US",2020-04-29
2894,4027,Yuma,Arizona,US,2020-04-30 02:32:27,32.768957,-113.906667,72,1,0,71,"Yuma, Arizona, US",2020-04-29
2895,8125,Yuma,Colorado,US,2020-04-30 02:32:27,40.003468,-102.425867,8,0,0,8,"Yuma, Colorado, US",2020-04-29
2896,48505,Zapata,Texas,US,2020-04-30 02:32:27,27.001564,-99.169872,7,0,0,7,"Zapata, Texas, US",2020-04-29


In [8]:
jhu_pivot = jhu_corona_df.pivot('FIPS', 'date').Confirmed
jhu_pivot

date,2020-04-12,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,2020-04-22,2020-04-23,2020-04-24,2020-04-25,2020-04-26,2020-04-27,2020-04-28,2020-04-29
FIPS,,,,,,,,,,,,,,,,,,
1001,19.0,19.0,23.0,24.0,26.0,26.0,25.0,26.0,28.0,30.0,32.0,33.0,36.0,36.0,37.0,39.0,40.0,43.0
1003,71.0,72.0,87.0,91.0,101.0,103.0,109.0,112.0,117.0,123.0,132.0,143.0,147.0,147.0,161.0,168.0,171.0,174.0
1005,10.0,10.0,11.0,12.0,14.0,15.0,18.0,20.0,22.0,28.0,29.0,30.0,32.0,32.0,33.0,35.0,37.0,37.0
1007,16.0,17.0,17.0,18.0,22.0,24.0,26.0,28.0,32.0,32.0,34.0,33.0,34.0,34.0,38.0,42.0,42.0,42.0
1009,13.0,14.0,16.0,17.0,18.0,20.0,20.0,21.0,22.0,26.0,29.0,31.0,31.0,31.0,34.0,34.0,34.0,36.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56035,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
56037,7.0,9.0,9.0,10.0,10.0,10.0,10.0,10.0,10.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,17.0
56039,56.0,56.0,57.0,58.0,59.0,61.0,62.0,62.0,62.0,92.0,93.0,93.0,95.0,95.0,95.0,95.0,96.0,96.0


In [9]:
#combine John Hopkins and NYT data, prioritizing John Hopkins
take_first = lambda s1, s2: s1 if s1 is not None else s2
corona_pivot = jhu_pivot.combine(nyt_pivot, take_first)
corona_pivot = corona_pivot[corona_pivot.index.isin(census_df.index)].bfill(axis = 1).ffill(axis = 1).dropna()
corona_pivot

date,2020-04-12,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,2020-04-22,2020-04-23,2020-04-24,2020-04-25,2020-04-26,2020-04-27,2020-04-28,2020-04-29
FIPS,,,,,,,,,,,,,,,,,,
1001,19.0,19.0,23.0,24.0,26.0,26.0,25.0,26.0,28.0,30.0,32.0,33.0,36.0,36.0,37.0,39.0,40.0,43.0
1003,71.0,72.0,87.0,91.0,101.0,103.0,109.0,112.0,117.0,123.0,132.0,143.0,147.0,147.0,161.0,168.0,171.0,174.0
1005,10.0,10.0,11.0,12.0,14.0,15.0,18.0,20.0,22.0,28.0,29.0,30.0,32.0,32.0,33.0,35.0,37.0,37.0
1007,16.0,17.0,17.0,18.0,22.0,24.0,26.0,28.0,32.0,32.0,34.0,33.0,34.0,34.0,38.0,42.0,42.0,42.0
1009,13.0,14.0,16.0,17.0,18.0,20.0,20.0,21.0,22.0,26.0,29.0,31.0,31.0,31.0,34.0,34.0,34.0,36.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56035,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
56037,7.0,9.0,9.0,10.0,10.0,10.0,10.0,10.0,10.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,17.0
56039,56.0,56.0,57.0,58.0,59.0,61.0,62.0,62.0,62.0,92.0,93.0,93.0,95.0,95.0,95.0,95.0,96.0,96.0


In [10]:
# calculate number of people infected everyday
corona_pivot_diff = corona_pivot.diff(axis=1)
corona_pivot_diff.dropna(axis=1, how='all', inplace=True)
corona_pivot_diff

date,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,2020-04-22,2020-04-23,2020-04-24,2020-04-25,2020-04-26,2020-04-27,2020-04-28,2020-04-29
FIPS,,,,,,,,,,,,,,,,,
1001,0.0,4.0,1.0,2.0,0.0,-1.0,1.0,2.0,2.0,2.0,1.0,3.0,0.0,1.0,2.0,1.0,3.0
1003,1.0,15.0,4.0,10.0,2.0,6.0,3.0,5.0,6.0,9.0,11.0,4.0,0.0,14.0,7.0,3.0,3.0
1005,0.0,1.0,1.0,2.0,1.0,3.0,2.0,2.0,6.0,1.0,1.0,2.0,0.0,1.0,2.0,2.0,0.0
1007,1.0,0.0,1.0,4.0,2.0,2.0,2.0,4.0,0.0,2.0,-1.0,1.0,0.0,4.0,4.0,0.0,0.0
1009,1.0,2.0,1.0,1.0,2.0,0.0,1.0,1.0,4.0,3.0,2.0,0.0,0.0,3.0,0.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56035,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
56037,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
56039,0.0,1.0,1.0,1.0,2.0,1.0,0.0,0.0,30.0,1.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0


In [11]:
# normalize by dividing by population
corona_pivot_diff_pct = corona_pivot_diff.apply(lambda x: x/[fip_pop_dict[i] for i in x.index.values])
corona_pivot_diff_pct

date,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,2020-04-22,2020-04-23,2020-04-24,2020-04-25,2020-04-26,2020-04-27,2020-04-28,2020-04-29
FIPS,,,,,,,,,,,,,,,,,
1001,0.000000,0.000072,0.000018,0.000036,0.000000,-0.000018,0.000018,0.000036,0.000036,0.000036,0.000018,0.000054,0.0,0.000018,0.000036,0.000018,0.000054
1003,0.000004,0.000067,0.000018,0.000045,0.000009,0.000027,0.000013,0.000022,0.000027,0.000040,0.000049,0.000018,0.0,0.000063,0.000031,0.000013,0.000013
1005,0.000000,0.000041,0.000041,0.000081,0.000041,0.000122,0.000081,0.000081,0.000243,0.000041,0.000041,0.000081,0.0,0.000041,0.000081,0.000081,0.000000
1007,0.000045,0.000000,0.000045,0.000179,0.000089,0.000089,0.000089,0.000179,0.000000,0.000089,-0.000045,0.000045,0.0,0.000179,0.000179,0.000000,0.000000
1009,0.000017,0.000035,0.000017,0.000017,0.000035,0.000000,0.000017,0.000017,0.000069,0.000052,0.000035,0.000000,0.0,0.000052,0.000000,0.000000,0.000035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56035,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000203,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
56037,0.000047,0.000000,0.000024,0.000000,0.000000,0.000000,0.000000,0.000000,0.000142,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000024
56039,0.000000,0.000043,0.000043,0.000043,0.000085,0.000043,0.000000,0.000000,0.001279,0.000043,0.000000,0.000085,0.0,0.000000,0.000000,0.000043,0.000000


In [12]:
# Calculate average change 

def func(x):
    change_pcts = [i for i in x if i != 0]
    avg = np.sum(change_pcts)/17
    
    # drop samples without too few changes or negative change
    if len(change_pcts) < 3 or avg <= 0:
        return float("NaN")
    
    return avg

average_row = lambda x: func(x)

betas = corona_pivot_diff_pct.T.apply(average_row).dropna()
beta_pcts = betas.describe()
display(beta_pcts)
display(betas)
fip_beta_dict = betas.to_dict()

count    2.028000e+03
mean     6.959331e-05
std      1.661591e-04
min      7.987118e-07
25%      1.305627e-05
50%      2.800493e-05
75%      6.697140e-05
max      3.364829e-03
dtype: float64

FIPS
1001     0.000025
1003     0.000027
1005     0.000064
1007     0.000068
1009     0.000023
           ...   
56025    0.000012
56037    0.000014
56039    0.000100
56041    0.000012
56043    0.000023
Length: 2028, dtype: float64

In [13]:
SIR_df = pd.read_csv('SIR.csv').set_index('FIPS')
SIR_df

,SUSCEPTIBLE,INFECTED,REMOVED
FIPS,,,
1001,0.999230,0.000340,0.000430
1003,0.999221,0.000372,0.000408
1005,0.998501,0.001013,0.000486
1007,0.998124,0.001072,0.000804
1009,0.999377,0.000329,0.000294
...,...,...,...
56037,0.999599,0.000165,0.000236
56039,0.995909,0.001620,0.002472
56041,0.999604,0.000198,0.000198


In [14]:
SIR_df_pct = SIR_df.rank(pct=True)
SIR_df_pct

,SUSCEPTIBLE,INFECTED,REMOVED
FIPS,,,
1001,0.402292,0.621579,0.592934
1003,0.398154,0.640993,0.572565
1005,0.229472,0.839911,0.637810
1007,0.183959,0.846276,0.780713
1009,0.474220,0.614577,0.469764
...,...,...,...
56037,0.614258,0.451623,0.398154
56039,0.065882,0.900382,0.950987
56041,0.619032,0.496181,0.350095


In [15]:
inf_dict = SIR_df_pct.INFECTED.to_dict()

def generate_beta(fip):
    try:
        return fip_beta_dict[fip]
    except KeyError:
        # Generate default beta based on infection rate percentile
        return inf_dict[fip]*(beta_pcts['75%']-beta_pcts['25%']) + beta_pcts['25%']
    
hyper_df = census_df.copy()
hyper_df['beta'] = [generate_beta(i) for i in hyper_df.index]
hyper_df['gamma'] = 1/14
hyper_df.drop(columns=['POPESTIMATE2019'], inplace=True)
hyper_df

,beta,gamma
FIPS,,
1001,0.000025,0.071429
1003,0.000027,0.071429
1005,0.000064,0.071429
1007,0.000068,0.071429
1009,0.000023,0.071429
...,...,...
56037,0.000014,0.071429
56039,0.000100,0.071429
56041,0.000012,0.071429


### Create hyperparam matrix and combine with other matrices

In [16]:
matrix_df = pd.concat([hyper_df, SIR_df], axis=1)
matrix_df['US'] = 1
display(matrix_df)
matrix_df.describe()

,beta,gamma,SUSCEPTIBLE,INFECTED,REMOVED,US
FIPS,,,,,,
1001,0.000025,0.071429,0.999230,0.000340,0.000430,1
1003,0.000027,0.071429,0.999221,0.000372,0.000408,1
1005,0.000064,0.071429,0.998501,0.001013,0.000486,1
1007,0.000068,0.071429,0.998124,0.001072,0.000804,1
1009,0.000023,0.071429,0.999377,0.000329,0.000294,1
...,...,...,...,...,...,...
56037,0.000014,0.071429,0.999599,0.000165,0.000236,1
56039,0.000100,0.071429,0.995909,0.001620,0.002472,1
56041,0.000012,0.071429,0.999604,0.000198,0.000198,1


,beta,gamma,SUSCEPTIBLE,INFECTED,REMOVED,US
count,3.142000e+03,3.142000e+03,3142.000000,3142.000000,3142.000000,3142.0
mean,5.404731e-05,7.142857e-02,0.998564,0.000695,0.000742,1.0
std,1.352539e-04,1.388000e-17,0.003507,0.002140,0.002029,0.0
min,7.987118e-07,7.142857e-02,0.898790,0.000000,0.000000,1.0
25%,1.887334e-05,7.142857e-02,0.998613,0.000045,0.000125,1.0
50%,2.391450e-05,7.142857e-02,0.999423,0.000202,0.000328,1.0
75%,4.592510e-05,7.142857e-02,0.999769,0.000604,0.000702,1.0
max,3.364829e-03,7.142857e-02,1.000000,0.053517,0.072636,1.0


In [17]:
matrix_dict = dict(zip(matrix_df.index, matrix_df.to_numpy()))
matrix_dict

{1001: array([2.52691959e-05, 7.14285714e-02, 9.99230342e-01, 3.40081262e-04,
        4.29576330e-04, 1.00000000e+00]),
 1003: array([2.71411323e-05, 7.14285714e-02, 9.99220549e-01, 3.71807162e-04,
        4.07643997e-04, 1.00000000e+00]),
 1005: array([6.43374906e-05, 7.14285714e-02, 9.98501175e-01, 1.01271976e-03,
        4.86105485e-04, 1.00000000e+00]),
 1007: array([6.82956044e-05, 7.14285714e-02, 9.98124498e-01, 1.07171564e-03,
        8.03786729e-04, 1.00000000e+00]),
 1009: array([2.33967623e-05, 7.14285714e-02, 9.99377443e-01, 3.28571923e-04,
        2.93985404e-04, 1.00000000e+00]),
 1011: array([4.65882819e-05, 7.14285714e-02, 9.98811999e-01, 3.96000396e-04,
        7.92000792e-04, 1.00000000e+00]),
 1013: array([1.30060251e-04, 7.14285714e-02, 9.97429042e-01, 2.10818593e-03,
        4.62772522e-04, 1.00000000e+00]),
 1015: array([1.76048589e-05, 7.14285714e-02, 9.99181374e-01, 2.72875314e-04,
        5.45750627e-04, 1.00000000e+00]),
 1017: array([1.68047011e-04, 7.14285714

In [18]:
county_np = np.loadtxt('county_matrix.csv', delimiter=',', dtype=int)
county_np.shape

(1792, 2944)

In [19]:
matrix = np.zeros([county_np.shape[0], county_np.shape[1], 6])

for i in range(county_np.shape[0]):
    for j in range(county_np.shape[1]):
        matrix[i, j] = matrix_dict.get(county_np[i, j], 0)
        
matrix.shape

(1792, 2944, 6)

In [20]:
np.save('matrix', matrix)